In [1]:
import pymysql
import dotenv # type: ignore
import os
import pymysql.cursors

dotenv.load_dotenv()

TABLE_NAME = 'customers'


connection = pymysql.connect(host=os.environ['MYSQL_HOST'],
                             user=os.environ['MYSQL_USER'],
                             password=os.environ['MYSQL_PASSWORD'],
                             database=os.environ['MYSQL_DATABASE'],
                             cursorclass=pymysql.cursors.DictCursor,)
# it's good to select now the database for you don't need to select which
# database and which table you are querying

# INSERT

In [2]:
with connection.cursor() as cursor:
    cursor.execute(
        f'CREATE TABLE IF NOT EXISTS {TABLE_NAME} ('
        'id INT NOT NULL AUTO_INCREMENT, '
        'name VARCHAR(50) NOT NULL, '
        'age INT NOT NULL, '
        'height REAL, '
        'PRIMARY KEY (id)'
        ') '
    )
        
    connection.commit()
    # import to commit every change in the database

In [3]:
with connection.cursor() as cursor:
    cursor.execute(
        f'INSERT INTO {TABLE_NAME} '
        '(name, age, height) VALUES ("Luís", 19, 182)'
    )
    cursor.execute(
        f'INSERT INTO {TABLE_NAME} '
        '(name, age) VALUES ("Edward", 20)'
    )
    connection.commit()

TRUNCATE TABLE {TABLE_NAME}

This code RESETS the table, use it carefully...

In [4]:
# other form (more recommended) to insert values in the database

sql = (f'INSERT INTO {TABLE_NAME} '
       '(name, age, height) '
       'VALUES '
       '(%s, %s, %s)'
       )

with connection.cursor() as cursor:
    cursor.execute(sql, ('Ian', 21, None))
    connection.commit()


In [5]:
with connection.cursor() as cursor:
    data = ('John', 54, None)
    cursor.execute(sql, data)
    connection.commit()

Every time we use a code line to execute a command using SQL codes, the security can be in danger.
So, to avoid it, it's preferred to use parameterized SQL queries, like this last insert using the 'sql' and 'data' var

In [6]:
sql2 = (f'INSERT INTO {TABLE_NAME} '
       '(name, age, height) '
       'VALUES '
       '(%(name)s, %(age)s, %(height)s)'
       )
# this variables between the () should follow the DICTS keys, not necessarily the columns names

with connection.cursor() as cursor:
    data2 = {
        'name': 'Olivia',
        'age': 19,
        'height': 155
    }
    
    cursor.execute(sql2, data2)
    connection.commit()

In [7]:
with connection.cursor() as cursor:
    data3 = ({'name': 'Lara', 'age': 18, 'height': 149},
             {'name': 'Rose', 'age': 44, 'height': None},
             {'name': 'Paul', 'age': 28, 'height': 193})
    cursor.executemany(sql2, data3)
    
    connection.commit()

# SELECT

In [8]:
with connection.cursor() as cursor:
    query = f'SELECT * FROM {TABLE_NAME} '
    
    cursor.execute(query)
    
    for row in cursor.fetchall():
        print(row)

{'id': 1, 'name': 'Luís', 'age': 19, 'height': 182.0}
{'id': 2, 'name': 'Edward', 'age': 20, 'height': None}
{'id': 3, 'name': 'Ian', 'age': 21, 'height': None}
{'id': 4, 'name': 'John', 'age': 54, 'height': None}
{'id': 5, 'name': 'Olivia', 'age': 19, 'height': 155.0}
{'id': 6, 'name': 'Lara', 'age': 18, 'height': 149.0}
{'id': 7, 'name': 'Rose', 'age': 44, 'height': None}
{'id': 8, 'name': 'Paul', 'age': 28, 'height': 193.0}


In [9]:
with connection.cursor() as cursor:
    query = f'SELECT * FROM {TABLE_NAME} WHERE height IS NULL'
    
    cursor.execute(query)
    # insecure form to do select
    
    for row in cursor.fetchall():
        print(row)

{'id': 2, 'name': 'Edward', 'age': 20, 'height': None}
{'id': 3, 'name': 'Ian', 'age': 21, 'height': None}
{'id': 4, 'name': 'John', 'age': 54, 'height': None}
{'id': 7, 'name': 'Rose', 'age': 44, 'height': None}


In [10]:
with connection.cursor() as cursor:
    column = 'id' # column hard-coded, prevents sql injection
    id_received = '5 AND id = 5' # can be an input, there's an exempt of atemp to do sql injection
    query = (f'SELECT * FROM {TABLE_NAME} '
            #  f'WHERE {column} < {id_received}' -> insecure
            f'WHERE {column} < %s ' # secure! Parameters prevents sql injection
             )
    
    cursor.execute(query, (id_received,))
    # insecure form to do select
    
    # the sql inject didn't worked!
    for row in cursor.fetchall():
        print(row)

{'id': 1, 'name': 'Luís', 'age': 19, 'height': 182.0}
{'id': 2, 'name': 'Edward', 'age': 20, 'height': None}
{'id': 3, 'name': 'Ian', 'age': 21, 'height': None}
{'id': 4, 'name': 'John', 'age': 54, 'height': None}


# DELETE

In [11]:
sql3 = (f'DELETE FROM {TABLE_NAME} '
        f'WHERE height IS NULL ')

with connection.cursor() as cursor:
        cursor.execute(sql3)
        connection.commit()
             
        cursor.execute(f'SELECT * FROM {TABLE_NAME}')
        for row in cursor.fetchall():
                print(row)
                

{'id': 1, 'name': 'Luís', 'age': 19, 'height': 182.0}
{'id': 5, 'name': 'Olivia', 'age': 19, 'height': 155.0}
{'id': 6, 'name': 'Lara', 'age': 18, 'height': 149.0}
{'id': 8, 'name': 'Paul', 'age': 28, 'height': 193.0}


# UPDATE

In [12]:
sql4 = (f'UPDATE {TABLE_NAME} '
        'SET name = %s, age = %s '
        'WHERE id = %s')

with connection.cursor() as cursor:
    data = ("Eleonor", 54, 5)
    cursor.execute(sql4, data)
    connection.commit()
    
    cursor.execute(f'SELECT * FROM {TABLE_NAME}')
    for row in cursor.fetchall():
        print(row)

{'id': 1, 'name': 'Luís', 'age': 19, 'height': 182.0}
{'id': 5, 'name': 'Eleonor', 'age': 54, 'height': 155.0}
{'id': 6, 'name': 'Lara', 'age': 18, 'height': 149.0}
{'id': 8, 'name': 'Paul', 'age': 28, 'height': 193.0}


SSCursor and SSDictCursor -> unbuffered
Very useful when working with queries which will return a lot of data, it stops in the middle of the query and asks if you want to continue, i.e they will fetch rows as needed, saving memory.

Limitations -> it can't return the total number of rows and can't scroll backward

In [13]:
connection.close()